In [1]:
import sys
from os.path import dirname, abspath
#_PARENT_DIR = dirname(dirname(dirname(abspath(__file__))))
#sys.path.append(_PARENT_DIR)
#from utils._utils import parse_celonis_url, get_user_data, get_logger              ### <--
from pycelonis import get_celonis
import argparse
#logger = get_logger()
def unify_tables(url, api_token, datapool, tables, tableSchema, function = 'null'):
    
    #loop through defined tables
    #for table_name in tables:
       
    c = get_celonis(url, api_token)
    #get_user_data(c, 'Table Unifier')
    #url_options= parse_celonis_url(url)                                            ### <--
    #pool = c.pools.find(url_options['id'])                                         ### <--
    
    pool = c.pools.find(datapool)                                                   ### <--

    minReq = []
    maxReq = []
    sameTables = []
    schemalist = tableSchema                                                        # added to extract required table schemas (in python list format)
    columns = ''
    select = ''
    
    # create or find global data job 
    try:
        global_dj = pool.data_jobs.find('Unify Tables')
    except:
        global_dj = pool.create_data_job(name = 'Unify Tables')
    
    #loop through defined tables
    for table_name in tables:
    
        # find tables to unify
        x = 0
        for schema in global_dj.tables:
            for table in schema:
                x = x + 1
                if table['schemaName'] not in schemalist:
                    continue
                if table['name'] == table_name:
                    sameTables.append(table)
        
        if len(sameTables) == 0:
            print('TABLE', table_name, 'CAN NOT BE UNIFIED! Not existing in source systems!')
            global_dj.create_transformation(name = "Unify " + table_name, statement = '-- no existing tables to unify in defined source systems')
            continue

        else:
            # create minimal column requirement
            for col in sameTables[0]['columns']:
                try:
                    if col in sameTables[1]['columns']:                                              # ERROR CHECKING !
                        minReq.append(col)
                except:
                    continue

            """
            #only required if function = 'min'

            if len(sameTables) > 2:
                for table in sameTables[2:]:
                    for col in minReq:
                        if col not in table['columns']:
                            minReq.pop(col)
                # create transformation statement for minimal table requirement

            if function == 'min':
                for col in minReq:
                    if(minReq.index(col) == 0):
                        columns += '{}\n'.format(col['name'])
                    else:
                        columns += ',{}\n'.format(col['name'])
                for table in sameTables:
                    select += 'SELECT \n{}FROM "{}"."{}"'.format(columns, table['schemaName'], table_name)
                    if table != sameTables[len(sameTables)-1]:
                        select += '\nUNION ALL\n'
            """            

            # create transformation for statement for extension with null
            if function == 'null':
                #columns = ''
                #select = ''
                
                for table in sameTables:
                    for col in table['columns']:
                        if col not in maxReq:
                            maxReq.append(col)
                for table in sameTables:
                    for col in maxReq:
                        if(maxReq.index(col) == 0):
                            columns += "'" + table['schemaName'] + "' " + 'AS "SCHEMA"\n' + ',{}\n'.format('"' + col['name'] + '"' if col in table['columns'] else 'NULL AS "' + col['name'] + '"')
                        else:
                            columns += ',{}\n'.format('"' + col['name'] + '"' if col in table['columns'] else 'NULL AS "' + col['name'] + '"')
                    select += 'SELECT \n{}FROM "{}"."{}"'.format(columns, table['schemaName'], table_name)
                    columns = ''
                    if table != sameTables[len(sameTables)-1]:
                        select += '\nUNION ALL\n'
            # create transformation
            #statement = 'DROP VIEW IF EXISTS "{}_UNIFIED";\n\nCreate VIEW "{}_UNIFIED" AS(\n{}\n);'.format(table_name, table_name, select)
            statement = 'DROP VIEW IF EXISTS "{}";\n\nCreate VIEW "{}" AS(\n{}\n);'.format(table_name, table_name, select)                      ### <-- Unified
            global_dj.create_transformation(name = "Unify " + table_name, statement = statement)
            print('The union statement has been saved in the global data job "Unify Tables" for Table', table_name)
            select = ''
            sameTables = []
            minReq = []
            maxReq = []
            
            # reminder: datatypes

In [2]:
# URL to the Cloud Data Pool where the tables to unify are hosted.
url = 'https://endresshauser.eu-3.celonis.cloud/'

# Specify a valid API token for your Cloud Team.
#Click on Team Settings -> Edit -> create or copy already inserted API key

api_token = 'NDEzZDFkN2YtZjVmZC00MTE0LTg2ODAtNmZlZDM5YzU5YTEwOjBxR0xOSFhjdDQ4MFNDc0V1UDIwbldzNWd2VXFVeUZieCt5MjdzbUhySkRP'


#To get the data pool id, you have to run the following code. To get the data
#the ML User here called Global needs permission to request these informations
#celonis = get_celonis()
#celonis.pools by that you get the ID for the data pool 

# Specify a datapool -- O2C Data Pool ID
datapool = '60442433-3dbb-4ce6-8ed3-e9bb7238c614'

# Specify the table names for which the tables should be merged. 
#KONV is not Celonis standard

tables =['T052U']

# Specify 'min' if the unified table should be the minimal column requirement and 'null' if the not existing columns should be filled with null values.
function = 'null'


#60442433-3dbb-4ce6-8ed3-e9bb7238c614_575c3915-e50d-48c0-a472-da19db3eee29 = PCA042
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_8ae9a322-9256-4298-bc90-2a6af6e6b36f = PIN020
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_195c34a6-cf16-431b-88f5-a608f44a7c14 = PIN024
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_3bbd20b8-81ce-4068-bc77-212ddfb300c5 = PP1011
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_e67ac4d6-dd7d-4653-a043-cdd83c9728b7 = PP2004
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_5bf5be9d-f13f-48fc-a5c2-0741f85727ae = PP2005
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_ff5eeb0b-b3ab-401f-8dc0-e701c4fdbd2a = PS1007
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_62c67a31-0987-4424-b967-64680da71015 = PS1030
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_d897dfda-15fb-4fa9-968f-90db8694e7cd = PS1032
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_acc25658-cfd5-4696-b4af-aa112af23fba = PS1033
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_9d804771-eac4-4fde-b23e-39710d5a8b58 = PS1035
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_80493315-33aa-43ce-a702-f970bab30f6f = PS1036
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_6567a63c-c54f-4edf-a594-a6100441211e  = PS1038
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_1cadd211-41b9-4632-a006-ffe0b103f034  = PS1039
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_6caea70a-87a4-41cc-bd4c-61c4829a6860 = PS1040
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_926ef30e-b9b4-4931-887b-a42794341bbf = PS1044
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_4d74b0c1-007e-4650-a312-46237a1d746c = PSG045
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_9b65d55f-cd2e-4aa7-89ad-af9fac513a95 = PSG048
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_d0b7b155-1fe2-4d29-89f2-5cfe11acdd05 = PSG049
#60442433-3dbb-4ce6-8ed3-e9bb7238c614_886a0dcb-393a-4184-abf7-fa6e53b6dc10 = PS1090

# Specify the required table schemas in a python list
#To acess the Schema Name you have to get the connection informations
#pool.data_connections
tableSchema = [
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_3bbd20b8-81ce-4068-bc77-212ddfb300c5',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_9d804771-eac4-4fde-b23e-39710d5a8b58',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_195c34a6-cf16-431b-88f5-a608f44a7c14',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_acc25658-cfd5-4696-b4af-aa112af23fba',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_ff5eeb0b-b3ab-401f-8dc0-e701c4fdbd2a',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_8ae9a322-9256-4298-bc90-2a6af6e6b36f',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_6567a63c-c54f-4edf-a594-a6100441211e',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_9b65d55f-cd2e-4aa7-89ad-af9fac513a95',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_5bf5be9d-f13f-48fc-a5c2-0741f85727ae',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_d0b7b155-1fe2-4d29-89f2-5cfe11acdd05',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_e67ac4d6-dd7d-4653-a043-cdd83c9728b7',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_6caea70a-87a4-41cc-bd4c-61c4829a6860',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_d897dfda-15fb-4fa9-968f-90db8694e7cd',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_926ef30e-b9b4-4931-887b-a42794341bbf',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_62c67a31-0987-4424-b967-64680da71015',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_4d74b0c1-007e-4650-a312-46237a1d746c',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_575c3915-e50d-48c0-a472-da19db3eee29',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_1cadd211-41b9-4632-a006-ffe0b103f034',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_80493315-33aa-43ce-a702-f970bab30f6f',
'60442433-3dbb-4ce6-8ed3-e9bb7238c614_886a0dcb-393a-4184-abf7-fa6e53b6dc10'    
]

unify_tables(url = url, api_token = api_token, datapool = datapool, tables = tables, tableSchema = tableSchema, function = function)

2020-10-20 10:26:38 - pycelonis: Login successful! Hello josef.rieger@cbs-consulting.de
The union statement has been saved in the global data job "Unify Tables" for Table T052U


In [3]:
url = 'https://endresshauser.eu-3.celonis.cloud/'

# Specify a valid API token for your Cloud Team.
#Click on Team Settings -> Edit -> create or copy already inserted API key

api_token = 'NDEzZDFkN2YtZjVmZC00MTE0LTg2ODAtNmZlZDM5YzU5YTEwOjBxR0xOSFhjdDQ4MFNDc0V1UDIwbldzNWd2VXFVeUZieCt5MjdzbUhySkRP'


#To get the data pool id, you have to run the following code. To get the data
#the ML User here called Global needs permission to request these informations
#celonis = get_celonis()
#celonis.pools by that you get the ID for the data pool 

# Specify a datapool -- O2C Data Pool ID
datapool = '60442433-3dbb-4ce6-8ed3-e9bb7238c614'

c = get_celonis()

2020-10-20 10:26:50 - pycelonis: Login successful! The Application Key currently has access to 25 Analyses and to 4 Data Pools.


In [4]:
  celonis = get_celonis(url, api_token)


2020-10-20 10:26:50 - pycelonis: Login successful! The Application Key currently has access to 25 Analyses and to 4 Data Pools.


In [5]:
pool = celonis.pools.find(datapool)

In [6]:
pool.data_connections

[<DataConnection, id 575c3915-e50d-48c0-a472-da19db3eee29, name SAP ECC - PCA-042>,
<DataConnection, id 3bbd20b8-81ce-4068-bc77-212ddfb300c5, name SAP ECC - PP1-011>,
<DataConnection, id 5bf5be9d-f13f-48fc-a5c2-0741f85727ae, name SAP ECC - PP2-005>,
<DataConnection, id 8ae9a322-9256-4298-bc90-2a6af6e6b36f, name SAP ECC - PIN-020>,
<DataConnection, id e67ac4d6-dd7d-4653-a043-cdd83c9728b7, name SAP ECC - PP2-004>,
<DataConnection, id 9b65d55f-cd2e-4aa7-89ad-af9fac513a95, name SAP ECC - PSG-048>,
<DataConnection, id 4d74b0c1-007e-4650-a312-46237a1d746c, name SAP ECC - PSG-045>,
<DataConnection, id d0b7b155-1fe2-4d29-89f2-5cfe11acdd05, name SAP ECC - PSG-049>,
<DataConnection, id ff5eeb0b-b3ab-401f-8dc0-e701c4fdbd2a, name SAP ECC - PS1-007>,
<DataConnection, id acc25658-cfd5-4696-b4af-aa112af23fba, name SAP ECC - PS1-033>,
<DataConnection, id 195c34a6-cf16-431b-88f5-a608f44a7c14, name SAP ECC - PIN-024>,
<DataConnection, id 62c67a31-0987-4424-b967-64680da71015, name SAP ECC - PS1-030>,
<Da